In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        pass
        #print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

<div style="text-align: center;">
      <h1>Görsel Açıdan Hurma</h1>

  <img src="https://blogger.googleusercontent.com/img/b/R29vZ2xl/AVvXsEgusCi9yUQpg6PZo2aOSoR3DwuTieJZ8Iyar5ils9PZYPqxvaaW29c4_9d83xcC2nrRDCwhvOg-CQH-xl-JCgZAUr4NythtePPt61FQG3RLrRdWaKk68aEs2HZsop6BWJwp8UXc67zV-cAQNKiz6ztNe0O0lVuV3U58d0VnNP77NbXK2j7f5oMyG1hKDZL3/s1040/Medjool%20Dates%20500grams.jpeg" alt="Hurma" width="700" height="500">
</div>


# Hurma CNN : Bu projemizde Hurmaların görsellerine bakara Convolitional Noural Network kullanarak onları sınıflandıracağız  

## İlk önce kaggle de bulunan bu hurma datalarını okumamız gerekiyor , bunun için bazı Python Kütüphanelerini kullanmamız gerekiyor 

In [ ]:
# Data işlemlerini DataFrame oluşturma ve okumak için pandas kütüphanesi
import pandas as pd 

# Sistem uyarı ve hatalarını görmezden gelmek için
import warnings
warnings.filterwarnings('ignore')

# Rsim okumak için 
import cv2 

# bilgisayardaki dosyalrarı okumak için 
import os # operating system 

In [ ]:
ls '/kaggle/input/date-fruit-image-dataset-in-controlled-environment'


### Burada gördüğünüz üzere yokardaki ilk kodun yolunu kullanarak altındaki değer klasörleri görüyoruz
#### Bu klasörler bizim Hurmaların Çeşitlerini gösteriyor   ,  bunları resimleri guruplandırmak için kullanıcağız  
#### Amacımız Train ve Test ederken  oluşturduğumuz Modelin Doğruluğunu hesaplamak olacak

# Datayı Oluşturma

In [ ]:
path='/kaggle/input/date-fruit-image-dataset-in-controlled-environment'
os.listdir(path)  # Gördüğünüz gibi Bizim dosyalarımı bu şekilde 
#  bu kullanım doğrudur   # Galaxy den başlıyor çünkü 

örnek:
/kaggle/input/date-fruit-image-dataset-in-controlled-environment/Galaxy/Galaxy Date (161).JPG

In [ ]:
# dosya yolumuz
path='/kaggle/input/date-fruit-image-dataset-in-controlled-environment'
# çeşitlerimiz
hurma_cesidi=['Galaxy','Rutab', 'Sugaey','Medjool','Nabtat Ali',
              'Ajwa','Sokari','Shaishe','Meneifi']


label_of_file=[]
img_list=[]

for kind in hurma_cesidi:
    kind_path = os.path.join(path, kind)
    
    for img in os.listdir(kind_path):
        
        img_list.append(os.path.join(kind_path, img))
        label_of_file.append(kind)
        

In [ ]:
# Verilen Değerleri Dataya Dönüştürme 

df=pd.DataFrame({'img':img_list , 'label':label_of_file})
df.sample(7)

# Verimizin Rastgele 5 tane görselin bakalım 

In [ ]:
import matplotlib.pyplot as plt
for index, row in df.sample(7).iterrows():
    # Resmi yükle
    img = cv2.imread(row['img'])
    
    # OpenCV resmi RGB formatına dönüştür
    img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    
    # Resmi matplotlib ile göster
    plt.figure(figsize=(6, 6))
    plt.imshow(img_rgb)
    plt.title(row['label'])
    plt.axis('off')
    plt.show()
    
    # Her resim için kısa bir bekleme süresi ekleyin
    #plt.pause(1)  # 1 saniye bekle

In [ ]:
df.info()  # data hakkında bilgi 

# Hurmaların Dağılımına bakalım  (meraktan)

In [ ]:
#pip install plotly

In [ ]:
import matplotlib.pyplot as plt
import plotly.express as px
sayi = df['label'].value_counts() #hurma türe göre sayisi 

# Plotly ile hurma dağılımını görselleştirme
fig=px.bar(x=sayi.index,  #6 indexi var
           y=sayi.values, # sayisi 
           color=sayi.index, # rekleri yine sayisina göre
           labels={'x': 'Çeşitler','y': 'Toplam Sayı'}, #label yazdırma
           title='Hurma Çeşitleri Grafiği', #başlık 
           template='plotly_dark') # arkapılan 

fig.show()

In [ ]:
sayi = df['label'].value_counts()
fig = px.pie(names=sayi.index,
             values=sayi.values,
             title='Hurma Dağılımı',
             labels={'names': 'Hurma',
                     'values': 'Toplam Sayı'},
             template='plotly_white')
fig.show()

# Datanın Label Değrlerini Encode Etme 

In [ ]:
from sklearn.preprocessing import LabelEncoder
encoder = LabelEncoder()

# Etiketleri sayısal değerlere dönüştür
df['encode_label'] = encoder.fit_transform(df['label'])

In [ ]:
df = df.sample(frac=1).reset_index(drop=True)

In [ ]:
df.sample(3)

# Normalizasyon

In [ ]:
import numpy as np
x=[]
for img in df['img']:
    try:
        img = cv2.imread(img)
        img = cv2.resize(img, (200, 200))# 448 pixle boyutunda
        img = img / 255.0  # normalize etmee 
        x.append(img)      # img 

    except:
        print(f"Error loading image: {img}")

# Model Oluşturma 
### datamızın x,y değerlerini belirleyeceğiz  sonra ise train ve test olarak ayıracağız


In [ ]:
x=np.array(x)
y=df['encode_label']

In [ ]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.25, random_state=42)

In [ ]:
x_train.shape,x_test.shape


# CNN Convolutional Netural Networks

## Bizim Yapacağmız işlemde bir nevi  bu olacak 
![CNN img](http://miro.medium.com/v2/resize:fit:1400/1*7_BCJFzekmPXmJQVRdDgwg.png)



### konu hakkında daha fazla bilgi için :
*  https://medium.com/@kocelifk/evri%C5%9Fimli-sinir-a%C4%9Flar%C4%B1na-convolutional-neural-networks-cnn-giri%C5%9F-7b4b4d5d07b3
*  https://medium.com/@kubra.ozcan_98680/convolutional-neural-networks-cnn-nedir-a3a9d5318835

In [ ]:
import tensorflow as tf
from tensorflow.keras import layers, models

In [ ]:
# Model oluşturma
model = models.Sequential()

# Evrişimsel katmanlar(Conv) ve havuzlama(pooling) katmanları

model.add(layers.Conv2D(32, (3, 3), activation='relu', input_shape=(200, 200, 3)))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Dropout(0.25))   # overfiting yapasın diye bazı değerleri atıyoruz dropout ile

model.add(layers.Conv2D(64, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Dropout(0.25))

model.add(layers.Conv2D(128, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Dropout(0.25))

model.add(layers.Conv2D(128, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Dropout(0.25))

model.add(layers.Conv2D(256, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Dropout(0.25))

# Düzleştirme ve tam bağlı katmanlar
model.add(layers.Flatten())  # düzleştirme

model.add(layers.Dense(512, activation='relu'))
model.add(layers.Dropout(0.5))  # Yüksek dropout oranı, tam bağlı katmanlarda aşırı öğrenmeyi azaltabilir
model.add(layers.Dense(9, activation='softmax'))  # Çıkış katmanı: 9 sınıf için # çeşit sayımıza göre 

# Modeli derleme
model.compile(optimizer='adam',loss='sparse_categorical_crossentropy',metrics=['accuracy'])


In [ ]:
# Model özetini yazdır
model.summary()

In [ ]:
history = model.fit(x_train, y_train, validation_data=(x_test, y_test), epochs=45, batch_size=150)

In [ ]:
# Saving Model
model.save('hurma_model.h5')

In [ ]:
# Eğitim ve Doğrulama Kaybı
plt.plot(history.history['loss'], label='Eğitim Kaybı')
plt.plot(history.history['val_loss'], label='Doğrulama Kaybı')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()
plt.show()

# Eğitim ve Doğrulama Doğruluğu
plt.plot(history.history['accuracy'], label='Eğitim Doğruluğu')
plt.plot(history.history['val_accuracy'], label='Doğrulama Doğruluğu')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend()
plt.show()

In [ ]:
loss,acc=model.evaluate(x_test,y_test)

In [ ]:
acc

In [ ]:
predict=model.predict(x_test)

In [ ]:
import seaborn as sns
from sklearn.metrics import f1_score,confusion_matrix,classification_report
confusion_matrix(y_test,np.argmax(predict,axis=1))

sns.heatmap(confusion_matrix(y_test,np.argmax(predict,axis=1)),annot=True)
plt.title('Confusion Matrix')

In [ ]:
print(classification_report(y_test,np.argmax(predict,axis=1)))

In [ ]:
f1_score(y_test, np.argmax(predict,axis=1), average='weighted')

# Dışardan alınan Göresle ile Resmi Tahmin etme 


In [ ]:
import requests
from PIL import Image
from io import BytesIO
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing.image import img_to_array


In [ ]:
# Geçerli bir görsel URL'si
#img_url = 'https://upload.wikimedia.org/wikipedia/commons/7/7e/Medjool-Date.jpg'
img_url='https://www.hurmashop.com/image//2021/mebrum_j_2021_001.jpg'

# Medjool

# Görseli URL'den indirin
response = requests.get(img_url)
img = Image.open(BytesIO(response.content))

# Görseli gösterme
plt.imshow(img)
plt.axis('off')  # Eksenleri kapat
plt.show()

In [ ]:
# Görseli yeniden boyutlandırma ve normalizasyon
def normalize_et(img):
    
    img = img.resize((200, 200))  # Modelin beklediği boyut
    img_array = img_to_array(img)
    img_array = img_array / 255.0  # Normalizasyon
    img_array = np.expand_dims(img_array, axis=0)  # Batch boyutu ekleyin
    return img_array

img_array=normalize_et(img)
# Model ile tahmin yapma
tahmin = model.predict(img_array)

tahim_sinifi = np.argmax(tahmin, axis=1) # kaç tane dğer var ise ona göre en yakını bulmaya çalışıyor 
#print(tahim_sinifi)

# Tahmin edilen sınıfı yazdırma
print(f"Tahmin edilen sınıf: {tahim_sinifi[0]}\nYani :  {df[df['encode_label'] == tahim_sinifi[0]]['label'].iloc[0]}")

# Sonuç.  CNN ile Modelimizi oluşturduk ve Gerekli olan değerleri verdik 
# bizim için 9 tane hurma çeşitinden internette aldığmız çeşiti tahmin etmeyi başardı 
# bir sonraki adım bunu hugieface ye app olarak yüklemek olacak  onun linki
> link:

  ## bir derin öğrenme modelini eğitim ve değerlendirme aşamalarında aşırı uyumu (overfitting) önlemek ve modelin genelleme yeteneğini artırmak için gerekli adımları içeren bir örnek sunar. 

In [ ]:
from tensorflow.keras.applications import VGG16
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten, Dropout, GlobalAveragePooling2D
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.model_selection import train_test_split

# Verileri eğitim ve doğrulama setlerine ayırma
X_train, X_val, y_train, y_val = train_test_split(x, y, test_size=0.2, random_state=42)

# Veri Augmentasyonu
datagen = ImageDataGenerator(
    rotation_range=30,
    width_shift_range=0.3,
    height_shift_range=0.3,
    shear_range=0.3,
    zoom_range=0.3,
    horizontal_flip=True,
    fill_mode='nearest'
)

datagen.fit(X_train)

# Transfer Öğrenme Modeli (VGG16)
base_model = VGG16(weights='imagenet', include_top=False, input_shape=(200, 200, 3))
model = Sequential([
    base_model,
    GlobalAveragePooling2D(),
    Dense(256, activation='relu'),
    Dropout(0.5),
    Dense(9, activation='softmax')
])

# Modeli derleme
optimizer = Adam(learning_rate=0.0001)
model.compile(optimizer=optimizer, loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Erken Durdurma
early_stopping = EarlyStopping(monitor='val_loss', patience=5)

# Modeli eğitim
history = model.fit(
    datagen.flow(X_train, y_train, batch_size=32),
    validation_data=(X_val, y_val),
    epochs=50,
    callbacks=[early_stopping]
)


başarı oranı 92

In [ ]:
# Saving Model
model.save('my_hurma_new_model.h5')

# Tansfer Learning

In [1]:
import tensorflow as tf
from tensorflow.keras.applications import VGG16
from tensorflow.keras.layers import GlobalAveragePooling2D, Dense, Dropout
from tensorflow.keras.models import Model
from tensorflow.keras.preprocessing.image import ImageDataGenerator


2024-08-05 05:42:15.689116: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-08-05 05:42:15.689220: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-08-05 05:42:15.821013: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [2]:
# Veri setini yükleme ve ön işleme
datagen = ImageDataGenerator(rescale=1./255, validation_split=0.2)

train_gen = datagen.flow_from_directory(
    

    '/kaggle/input/date-fruit-image-dataset-in-controlled-environment',  # Buraya veri setinin bulunduğu klasörün tam yolunu yazın
    target_size=(150, 150),
    batch_size=32,
    class_mode='categorical',
    subset='training'
)

val_gen = datagen.flow_from_directory(
    '/kaggle/input/date-fruit-image-dataset-in-controlled-environment',  # Buraya veri setinin bulunduğu klasörün tam yolunu yazın
    target_size=(150, 150),
    batch_size=32,
    class_mode='categorical',
    subset='validation'
)


Found 1329 images belonging to 9 classes.
Found 329 images belonging to 9 classes.


In [3]:
# VGG16 modelini yükleme ve son katmanlarını çıkarma
base_model = VGG16(weights='imagenet', include_top=False, input_shape=(150, 150, 3))

# Yeni katmanlar ekleme
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(512, activation='relu')(x)
x = Dropout(0.5)(x)
predictions = Dense(train_gen.num_classes, activation='softmax')(x)

# Yeni model oluşturma
model_vgg16 = Model(inputs=base_model.input, outputs=predictions)

# Temel modelin ağırlıklarını dondurma
for layer in base_model.layers:
    layer.trainable = False


58889256/58889256 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step


In [4]:
# Modeli derleme
model_vgg16.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Modeli eğitme
model_vgg16.fit(train_gen, epochs=10, validation_data=val_gen)


Epoch 1/10


/opt/conda/lib/python3.10/site-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()
2024-08-05 05:43:00.620918: E external/local_xla/xla/service/gpu/buffer_comparator.cc:1137] Difference at 0: 4.18794, expected 3.54228
2024-08-05 05:43:00.620982: E external/local_xla/xla/service/gpu/buffer_comparator.cc:1137] Difference at 3: 5.14647, expected 4.50081
2024-08-05 05:43:00.620996: E external/local_xla/xla/service/gpu/buffer_comparator.cc:1137] Difference at 8: 5.30074, expected 4.65509
2024-08-05 05:43:00.621010: E external/local_xla/xla/service/gpu/buffer_comparator.cc:1137] Difference at 9: 4.91575, expected 4.27009
2024-08-05 05:43:00.621027: E external/local_xla/xla/service/gpu/buffer_comparator.cc:11

 1/42 ━━━━━━━━━━━━━━━━━━━━ 24:52 36s/step - accuracy: 0.1562 - loss: 2.7533

I0000 00:00:1722836589.883919     159 device_compiler.h:186] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


24/42 ━━━━━━━━━━━━━━━━━━━━ 52s 3s/step - accuracy: 0.1506 - loss: 2.3858

2024-08-05 05:44:20.867573: E external/local_xla/xla/service/gpu/buffer_comparator.cc:1137] Difference at 45000: 4.2434, expected 3.38296
2024-08-05 05:44:20.867633: E external/local_xla/xla/service/gpu/buffer_comparator.cc:1137] Difference at 45001: 5.8421, expected 4.98165
2024-08-05 05:44:20.867643: E external/local_xla/xla/service/gpu/buffer_comparator.cc:1137] Difference at 45002: 6.49631, expected 5.63587
2024-08-05 05:44:20.867651: E external/local_xla/xla/service/gpu/buffer_comparator.cc:1137] Difference at 45003: 5.704, expected 4.84356
2024-08-05 05:44:20.867659: E external/local_xla/xla/service/gpu/buffer_comparator.cc:1137] Difference at 45004: 6.42639, expected 5.56595
2024-08-05 05:44:20.867667: E external/local_xla/xla/service/gpu/buffer_comparator.cc:1137] Difference at 45005: 6.23804, expected 5.3776
2024-08-05 05:44:20.867674: E external/local_xla/xla/service/gpu/buffer_comparator.cc:1137] Difference at 45006: 7.35888, expected 6.49844
2024-08-05 05:44:20.867682: E ex

41/42 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step - accuracy: 0.1953 - loss: 2.2637

2024-08-05 05:45:25.071460: E external/local_xla/xla/service/gpu/buffer_comparator.cc:1137] Difference at 0: 3.41052, expected 2.94401
2024-08-05 05:45:25.071517: E external/local_xla/xla/service/gpu/buffer_comparator.cc:1137] Difference at 11: 3.42955, expected 2.96304
2024-08-05 05:45:25.071531: E external/local_xla/xla/service/gpu/buffer_comparator.cc:1137] Difference at 57: 3.19639, expected 2.72988
2024-08-05 05:45:25.071560: E external/local_xla/xla/service/gpu/buffer_comparator.cc:1137] Difference at 58: 3.56411, expected 3.0976
2024-08-05 05:45:25.071574: E external/local_xla/xla/service/gpu/buffer_comparator.cc:1137] Difference at 146: 3.49676, expected 3.03024
2024-08-05 05:45:25.071638: E external/local_xla/xla/service/gpu/buffer_comparator.cc:1137] Difference at 8400: 3.60682, expected 3.1403
2024-08-05 05:45:25.071655: E external/local_xla/xla/service/gpu/buffer_comparator.cc:1137] Difference at 9450: 3.37633, expected 2.90982
2024-08-05 05:45:25.071669: E external/local_x

42/42 ━━━━━━━━━━━━━━━━━━━━ 175s 3s/step - accuracy: 0.2010 - loss: 2.2506 - val_accuracy: 0.6505 - val_loss: 1.4623
Epoch 2/10
42/42 ━━━━━━━━━━━━━━━━━━━━ 114s 2s/step - accuracy: 0.5385 - loss: 1.4542 - val_accuracy: 0.7538 - val_loss: 1.1127
Epoch 3/10
42/42 ━━━━━━━━━━━━━━━━━━━━ 117s 2s/step - accuracy: 0.6434 - loss: 1.2082 - val_accuracy: 0.7994 - val_loss: 0.8370
Epoch 4/10
42/42 ━━━━━━━━━━━━━━━━━━━━ 124s 3s/step - accuracy: 0.7277 - loss: 0.8939 - val_accuracy: 0.7629 - val_loss: 0.7425
Epoch 5/10
42/42 ━━━━━━━━━━━━━━━━━━━━ 111s 2s/step - accuracy: 0.7118 - loss: 0.8957 - val_accuracy: 0.8298 - val_loss: 0.6494
Epoch 6/10
42/42 ━━━━━━━━━━━━━━━━━━━━ 115s 2s/step - accuracy: 0.7616 - loss: 0.7394 - val_accuracy: 0.7933 - val_loss: 0.6345
Epoch 7/10
42/42 ━━━━━━━━━━━━━━━━━━━━ 118s 2s/step - accuracy: 0.7924 - loss: 0.6593 - val_accuracy: 0.8389 - val_loss: 0.5849
Epoch 8/10
42/42 ━━━━━━━━━━━━━━━━━━━━ 144s 2s/step - accuracy: 0.7878 - loss: 0.6425 - val_accuracy: 0.8328 - val_loss: 0.

In [5]:
# Modeli kaydetme
model_vgg16.save('date_fruit_vgg16_model.h5')
